In [56]:
import pandas as pd
import numpy as np
import re
import nltk
import emoji
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from spellchecker import SpellChecker
#Due to time constraint/Compute constraints will do in sample and the test for fulldatset
data=pd.read_csv("C:/Users/AdityaBhatt/Desktop/sample.csv")
data=data[['text']]
data.head()
    

,text
0,@AppleSupport causing the reply to be disregar...
1,@105835 Your business means a lot to us. Pleas...
2,@76328 I really hope you all change but I'm su...
3,@105836 LiveChat is online at the moment - htt...
4,@VirginTrains see attached error message. I've...


In [57]:
class TextPreprocessing:
    def __init__(self, data):
        self.data = data
        self.stop_words = set(stopwords.words('english'))
        self.stemmer = PorterStemmer()
        self.lemmatizer = WordNetLemmatizer()
        self.spell_checker = SpellChecker()

    def clean_text(self, text):
        stop_words = set(stopwords.words('english'))
        
        
        # Dictionary of common abbreviations and their full forms
        abbreviations = {
            "u": "you",
            "r": "are",
            "thx": "thanks",
            "lol": "laugh out loud",
            "brb": "be right back",
            "btw": "by the way",
            "afaik": "as far as I know",
            "imho": "in my humble opinion",
            "omg": "oh my god",
            "fyi": "for your information",
            "tbh": "to be honest",
            "idk": "I don't know",
            "yolo": "you only live once",
            "gtg": "got to go",
            "hmu": "hit me up",
            "tmi": "too much information",
            "bff": "best friends forever",
            "icymi": "in case you missed it",
            "irl": "in real life",
            "fomo": "fear of missing out",
            "np": "no problem",
            "omw": "on my way",
            "smh": "shaking my head",
            "rofl": "rolling on the floor laughing",
            "imo": "in my opinion",
            "tbt": "throwback Thursday",
            "wcw": "woman crush Wednesday",
            "mcm": "man crush Monday",
            "ootd": "outfit of the day",
            "hbd": "happy birthday",
            "tgif": "thank goodness it's Friday",
            "nsfw": "not safe for work",
            "btw": "by the way",
            "afaik": "as far as I know",
            "icymi": "in case you missed it",
            "yolo": "you only live once",
            "fomo": "fear of missing out",
            "tmi": "too much information",
            "imo": "in my opinion",
            # Add more abbreviations as needed
        }
        # Remove HTML tags
        text = re.sub(r'<.*?>', '', text)
    
        # Remove special characters and numbers
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        
        # Tokenization
        tokens = word_tokenize(text)
        text = ' '.join(tokens)
        
        # Remove stopwords
        tokens = [word for word in tokens if word.lower() not in stop_words]
        # Replace abbreviations
        tokens = [abbreviations.get(word, word) for word in tokens]
        # Stemming
        tokens = [self.stemmer.stem(word) for word in tokens]
        #Lemmatization
        tokens = [self.lemmatizer.lemmatize(word) for word in tokens]
        
        # Spelling correction
        corrected_tokens = [self.spell_checker.correction(word) if word is not None else None for word in tokens]
        corrected_tokens = [word for word in corrected_tokens if word is not None]
        
        text = ' '.join(corrected_tokens)
        
        
        text = ' '.join(tokens)
        
        # Convert to lowercase
        text = text.lower()
        
        # Convert to uppercase (optional)
        # text = text.upper()
        
         # Remove emojis
        text = emoji.demojize(text)
        
        # Remove punctuation
        text = re.sub(r'[^\w\s]', '', text)
        
        # Remove URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
        
         # Handle white spaces
        text = re.sub(r'\s+', ' ', text).strip()
    
        return text
        
        

    def apply_cleaning(self):
        # Apply the clean_text function to the 'text' column
        self.data['cleaned_text'] = self.data['text'].apply(self.clean_text)
        
        return self.data


clean_data=TextPreprocessing(data)
clean_data=pd.DataFrame(clean_data.apply_cleaning())        

In [58]:
clean_data=clean_data[['cleaned_text']]

In [59]:
clean_data

,cleaned_text
0,applesupport caus repli disregard tap notif ke...
1,busi mean lot u plea dm name zip code addit de...
2,realli hope chang im sure wont dont
3,livechat onlin moment contact option leav mess...
4,virgintrain see attach error messag ive tri le...
...,...
88,wish amazon option get ship up store avoid lot...
89,reschedul shit tomorrow
90,hey sara sorri hear issu ask lay speed websit ...
91,tesco bit find layout cumbersom remov item fav...


# One hot encoding

In [61]:
from sklearn.feature_extraction.text import CountVectorizer

# Create an instance of CountVectorizer
vectorizer = CountVectorizer(binary=True)

# Fit and transform the 'cleaned_text' column
one_hot_encoded = vectorizer.fit_transform(clean_data['cleaned_text']).toarray()

# Create a DataFrame with the one-hot encoded features
one_hot_df = pd.DataFrame(one_hot_encoded, columns=vectorizer.get_feature_names_out())

# Concatenate the one-hot encoded features with the original DataFrame
clean_data_encoded =pd.DataFrame(pd.concat([clean_data, one_hot_df], axis=1))

# Display the resulting DataFrame
print(clean_data_encoded)


                                         cleaned_text  ac  account  ad  addit  \
0   applesupport caus repli disregard tap notif ke...   0        0   0      0   
1   busi mean lot u plea dm name zip code addit de...   0        0   0      1   
2                 realli hope chang im sure wont dont   0        0   0      0   
3   livechat onlin moment contact option leav mess...   0        0   0      0   
4   virgintrain see attach error messag ive tri le...   0        0   0      0   
..                                                ...  ..      ...  ..    ...   
88  wish amazon option get ship up store avoid lot...   0        0   0      0   
89                            reschedul shit tomorrow   0        0   0      0   
90  hey sara sorri hear issu ask lay speed websit ...   0        0   0      0   
91  tesco bit find layout cumbersom remov item fav...   0        0   0      0   
92  doesnt help plea dm full name address email in...   0        0   0      0   

    address  adnam  advis  

# BOW

In [62]:
# Create an instance of CountVectorizer
vectorizer = CountVectorizer(binary=True)

# Fit and transform the 'cleaned_text' column
one_hot_encoded = vectorizer.fit_transform(clean_data['cleaned_text']).toarray()

# Create a DataFrame with the one-hot encoded features
one_hot_df = pd.DataFrame(one_hot_encoded, columns=vectorizer.get_feature_names_out())

# Concatenate the one-hot encoded features with the original DataFrame
clean_data_encoded = pd.concat([clean_data, one_hot_df], axis=1)

# Display the resulting DataFrame
print(clean_data_encoded)


                                         cleaned_text  ac  account  ad  addit  \
0   applesupport caus repli disregard tap notif ke...   0        0   0      0   
1   busi mean lot u plea dm name zip code addit de...   0        0   0      1   
2                 realli hope chang im sure wont dont   0        0   0      0   
3   livechat onlin moment contact option leav mess...   0        0   0      0   
4   virgintrain see attach error messag ive tri le...   0        0   0      0   
..                                                ...  ..      ...  ..    ...   
88  wish amazon option get ship up store avoid lot...   0        0   0      0   
89                            reschedul shit tomorrow   0        0   0      0   
90  hey sara sorri hear issu ask lay speed websit ...   0        0   0      0   
91  tesco bit find layout cumbersom remov item fav...   0        0   0      0   
92  doesnt help plea dm full name address email in...   0        0   0      0   

    address  adnam  advis  

# N GRAM

In [63]:
# Create instances of CountVectorizer for n-grams of lengths 2, 3, and 4
vectorizer_2grams = CountVectorizer(ngram_range=(2, 2))
vectorizer_3grams = CountVectorizer(ngram_range=(3, 3))
vectorizer_4grams = CountVectorizer(ngram_range=(4, 4))

# Fit and transform the 'cleaned_text' column for each n-gram length
ngrams_2grams = vectorizer_2grams.fit_transform(clean_data['cleaned_text']).toarray()
ngrams_3grams = vectorizer_3grams.fit_transform(clean_data['cleaned_text']).toarray()
ngrams_4grams = vectorizer_4grams.fit_transform(clean_data['cleaned_text']).toarray()

# Create DataFrames with the n-gram features for each length
ngrams_df_2grams = pd.DataFrame(ngrams_2grams, columns=vectorizer_2grams.get_feature_names_out())
ngrams_df_3grams = pd.DataFrame(ngrams_3grams, columns=vectorizer_3grams.get_feature_names_out())
ngrams_df_4grams = pd.DataFrame(ngrams_4grams, columns=vectorizer_4grams.get_feature_names_out())

# Concatenate the n-gram features with the original DataFrame
clean_data_ngrams = pd.concat([clean_data, ngrams_df_2grams, ngrams_df_3grams, ngrams_df_4grams], axis=1)

# Display the resulting DataFrame
print(clean_data_ngrams)

                                         cleaned_text  account as  \
0   applesupport caus repli disregard tap notif ke...           0   
1   busi mean lot u plea dm name zip code addit de...           0   
2                 realli hope chang im sure wont dont           0   
3   livechat onlin moment contact option leav mess...           0   
4   virgintrain see attach error messag ive tri le...           0   
..                                                ...         ...   
88  wish amazon option get ship up store avoid lot...           0   
89                            reschedul shit tomorrow           0   
90  hey sara sorri hear issu ask lay speed websit ...           0   
91  tesco bit find layout cumbersom remov item fav...           0   
92  doesnt help plea dm full name address email in...           0   

    account inform  account prior  addit detail  address email  address name  \
0                0              0             0              0             0   
1          

# TFIDF

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Create an instance of TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the 'cleaned_text' column
tfidf_features = tfidf_vectorizer.fit_transform(clean_data['cleaned_text']).toarray()

# Create a DataFrame with the TF-IDF features
tfidf_df = pd.DataFrame(tfidf_features, columns=tfidf_vectorizer.get_feature_names_out())

# Concatenate the TF-IDF features with the original DataFrame
clean_data_tfidf = pd.concat([clean_data, tfidf_df], axis=1)

# Display the resulting DataFrame
print(clean_data_tfidf)

                                         cleaned_text   ac  account   ad  \
0   applesupport caus repli disregard tap notif ke...  0.0      0.0  0.0   
1   busi mean lot u plea dm name zip code addit de...  0.0      0.0  0.0   
2                 realli hope chang im sure wont dont  0.0      0.0  0.0   
3   livechat onlin moment contact option leav mess...  0.0      0.0  0.0   
4   virgintrain see attach error messag ive tri le...  0.0      0.0  0.0   
..                                                ...  ...      ...  ...   
88  wish amazon option get ship up store avoid lot...  0.0      0.0  0.0   
89                            reschedul shit tomorrow  0.0      0.0  0.0   
90  hey sara sorri hear issu ask lay speed websit ...  0.0      0.0  0.0   
91  tesco bit find layout cumbersom remov item fav...  0.0      0.0  0.0   
92  doesnt help plea dm full name address email in...  0.0      0.0  0.0   

       addit  address  adnam  advis  affect  age  ...  work  wtf  xfiniti  \
0   0.0000

# ONE HOT ENCODING VS BOW VS N GRAM VS TFIDF

Certainly! Here are five advantages and five disadvantages for each of the following text representation techniques: One-Hot Encoding (OHE), Bag of Words (BoW), N-grams, and Term Frequency-Inverse Document Frequency (TF-IDF).

### One-Hot Encoding (OHE):

**Advantages:**

1. **Simple Representation:** OHE is a straightforward and easy-to-understand representation, where each word is represented by a binary value (0 or 1).

2. **No Information Loss:** It maintains the information about the presence or absence of words in a document without losing any information.

3. **Independence:** OHE treats each word independently, making it suitable for certain tasks where word order is less important.

4. **No Weighting:** There is no weighting of words, making it useful when equal importance is assigned to all words.

5. **Interpretability:** The resulting matrix is interpretable, making it easy to inspect and understand the features.

**Disadvantages:**

1. **High Dimensionality:** OHE results in a high-dimensional sparse matrix, especially when dealing with a large vocabulary, which can be computationally expensive.

2. **Lack of Semantic Information:** OHE does not capture semantic relationships between words; it treats all words as independent features.

3. **No Contextual Information:** It doesn't consider the context in which words appear, making it less effective in capturing meaning.

4. **Memory Consumption:** The high number of features can lead to increased memory consumption.

5. **Not Suitable for Large Texts:** OHE may not be practical for large texts due to the resulting large and sparse feature space.

### Bag of Words (BoW):

**Advantages:**

1. **Simplicity:** BoW is simple and easy to implement, making it a good baseline representation.

2. **No Prior Knowledge Required:** BoW doesn't require prior knowledge of the language or domain-specific information.

3. **Flexibility:** It can be applied to various NLP tasks, including sentiment analysis, classification, and clustering.

4. **Efficient Storage:** The sparse nature of the BoW representation makes it memory-efficient.

5. **Interpretability:** Similar to OHE, the resulting matrix is interpretable.

**Disadvantages:**

1. **Loss of Word Order:** BoW discards the order of words, losing some syntactic and semantic information.

2. **Equal Weight for All Words:** It treats all words equally, ignoring variations in importance.

3. **High Dimensionality:** Similar to OHE, BoW can result in a high-dimensional feature space.

4. **No Semantic Information:** BoW lacks the ability to capture semantic relationships between words.

5. **Difficulty with Synonyms:** It struggles to capture synonym relationships, as each word is treated independently.

### N-grams:

**Advantages:**

1. **Captures Local Context:** N-grams capture local context and word dependencies, providing more information about word relationships.

2. **Flexible Context Modeling:** N-grams allow flexibility in capturing context by considering sequences of adjacent words.

3. **Improved Representation:** They can capture partial word ordering and provide a more detailed representation of the text.

4. **Contextual Information:** N-grams can capture some level of contextual information, especially with larger values of N.

5. **Effective for Short Texts:** N-grams can be effective for short texts or when word order is important.

**Disadvantages:**

1. **Increased Dimensionality:** As N increases, the dimensionality of the feature space grows exponentially.

2. **Sparse Data:** For larger values of N, the data can become sparse, leading to increased computational requirements.

3. **Loss of Global Context:** N-grams may lose global context and struggle to capture long-range dependencies.

4. **Not Suitable for All Tasks:** While effective for certain tasks, N-grams may not be suitable for tasks where global context is crucial.

5. **Data Sparsity:** Sparse data can be challenging to handle and may require additional techniques.

### TF-IDF (Term Frequency-Inverse Document Frequency):

**Advantages:**

1. **Weighting of Words:** TF-IDF introduces term weighting, considering both term frequency and inverse document frequency.

2. **Contextual Information:** It captures contextual information by assigning higher weights to important terms.

3. **Reduced Impact of Stopwords:** Common words (stopwords) are down-weighted, reducing their impact on the representation.

4. **Global Context:** TF-IDF considers both local and global context, providing a balance between the two.

5. **Effective for Long Documents:** TF-IDF can be effective for representing long documents where word order may be less critical.

**Disadvantages:**

1. **Complexity:** TF-IDF introduces additional complexity compared to simpler representations like BoW.

2. **Difficulty with Synonyms:** It may still struggle with synonyms, as the focus is on term frequency and document frequency.

3. **Loss of Word Order:** Like BoW, TF-IDF doesn't capture word order information.

4. **Not Suitable for Short Texts:** In some cases, TF-IDF may not perform well on very short texts.

5. **Requires Document Frequency Information:** Document frequency information is needed, which may not be readily available for all texts.
